# AUV Localization Using Dead Reckoning Techniques with IMU Sensor

In [2]:
#Load Libraries
import numpy as np            
import pandas as pd           
import matplotlib.pyplot as plt  
import scipy                  
import filterpy           

In [3]:
#Calculate Orientation

def calculate_orientation(a_x, a_y, a_z, previous_pitch, previous_roll):
    
    alpha = 0.9999
    
    theta_z = np.arctan2(a_y, a_z)
    theta_x = np.arctan2(a_x, np.sqrt(a_y**2 + a_z**2))
    pitch = (previous_pitch * alpha) + (theta_x * (1 - alpha))
    roll = (previous_roll * alpha) + (theta_z * (1 - alpha))
    
    return pitch, roll

In [5]:
#Calculate Velocity

def calculate_velocity(a_x, a_y, a_z, previous_velocity_x, previous_velocity_y, previous_velocity_z, delta_t):
    
    velocity_x = previous_velocity_x + a_x * delta_t
    velocity_y = previous_velocity_y + a_y * delta_t
    velocity_z = previous_velocity_z + a_z * delta_t
    
    return velocity_x, velocity_y, velocity_z 

In [6]:
#Calculate Position

def calculate_position(previous_position_x, previous_position_y, velocity_x, velocity_y, delta_t):
    
    position_x = previous_position_x + velocity_x * delta_t
    position_y = previous_position_y + velocity_y * delta_t

    return new_position_x, new_position_y

In [ ]:
from filterpy.kalman import ExtendedKalmanFilter



In [ ]:
# Initialize variables
previous_position_x_1, previous_position_y_1 = 0, 0
previous_position_x_2, previous_position_y_2 = 0, 0
previous_position_x_b, previous_position_y_b = 0, 0

previous_velocity_x_1, previous_velocity_y_1, previous_velocity_z_1 = 0, 0, 0
previous_velocity_x_2, previous_velocity_y_2, previous_velocity_z_2 = 0, 0, 0
previous_velocity_x_b, previous_velocity_y_b, previous_velocity_z_b = 0, 0, 0

previous_pitch_1, previous_roll_1 = 0, 0
previous_pitch_2, previous_roll_2 = 0, 0
previous_pitch_b, previous_roll_b = 0, 0

delta_t = 0.1  # Time step

for i in range(len(imu_data_1)):
    
    filtered_data_1 = ekf(imu_data_1[i])
    filtered_data_2 = ekf(imu_data_2[i])
    filtered_data_b = ekf_blend(imu_data_1[i], imu_data_2[i])
    
    #Calculate DR for IMU 1
    a_x_1, a_y_1, a_z_1 = filtered_data_1
    pitch_1, roll_1 = calculate_orientation(a_x_1, a_y_1, a_z_1, previous_pitch_1, previous_roll_1)
    velocity_x_1, velocity_y_1, velocity_z_1 = calculate_velocity(a_x_1, a_y_1, a_z_1, previous_velocity_x_1, previous_velocity_y_1, previous_velocity_z_1, delta_t)
    position_x_1, position_y_1 = calculate_position(previous_position_x_1, previous_position_y_1, velocity_x_1, velocity_y_1, delta_t)

    #Calculate DR for IMU 2
    a_x_2, a_y_2, a_z_2 = filtered_data_2
    pitch_2, roll_2 = calculate_orientation(a_x_2, a_y_2, a_z_2, previous_pitch_2, previous_roll_2)
    velocity_x_2, velocity_y_2, velocity_z_2 = calculate_velocity(a_x_2, a_y_2, a_z_2, previous_velocity_x_2, previous_velocity_y_2, previous_velocity_z_2, delta_t)
    position_x_2, position_y_2 = calculate_position(previous_position_x_2, previous_position_y_2, velocity_x_2, velocity_y_2, delta_t)
    
    #Calculate DR for blended data
    a_x_b, a_y_b, a_z_b = filtered_data_b
    pitch_b, roll_b = calculate_orientation(a_x_b, a_y_b, a_z_b, previous_pitch_b, previous_roll_b)
    velocity_x_b, velocity_y_b, velocity_z_b = calculate_velocity(a_x_b, a_y_b, a_z_b, previous_velocity_x_b, previous_velocity_y_b, previous_velocity_z_b, delta_t)
    position_x_b, position_y_b = calculate_position(previous_position_x_b, previous_position_y_b, velocity_x_b, velocity_y_b, delta_t)
    
    # Update the previous values for IMU 1
    previous_pitch_1, previous_roll_1 = pitch_1, roll_1
    previous_velocity_x_1, previous_velocity_y_1, previous_velocity_z_1 = velocity_x_1, velocity_y_1, velocity_z_1
    previous_position_x_1, previous_position_y_1 = position_x_1, position_y_1
    
    # Update the previous values for IMU 2
    previous_pitch_2, previous_roll_2 = pitch_2, roll_2
    previous_velocity_x_2, previous_velocity_y_2, previous_velocity_z_2 = velocity_x_2, velocity_y_2, velocity_z_2
    previous_position_x_2, previous_position_y_2 = position_x_2, position_y_2
    
    # Update the previous values for blended data
    previous_pitch_b, previous_roll_b = pitch_b, roll_b
    previous_velocity_x_b, previous_velocity_y_b, previous_velocity_z_b = velocity_x_b, velocity_y_b, velocity_z_b
    previous_position_x_b, previous_position_y_b = position_x_b, position_y_b